# 1. Background Problem (20%)
Language modeling is a fundamental task in Natural Language Processing (NLP), used in various applications like predictive typing, text generation, and spelling correction. For this project, I chose the Sci-Fi Stories Text Corpus available on Kaggle. Sci-Fi literature is linguistically rich and imaginative, often pushing boundaries of vocabulary and structure. Modeling such text is both challenging and rewarding, and it provides an exciting opportunity to explore how well statistical language models and autocorrect systems can handle complex and creative writing. Recent research has demonstrated that large language models can generalize to a wide variety of tasks, including creative text generation and spelling correction, even in few-shot settings. Studies have also shown that while language models are capable of producing creative writing, they face unique challenges in maintaining coherence and handling the imaginative language found in genres like science fiction. Furthermore, advances in spelling correction techniques have highlighted the importance of robust language modeling for correcting errors in creative and domain-specific texts

References:
* Brown, T. B., et al. (2020). Language Models are Few-Shot Learners.
* Clark, E., et al. (2021). The Effectiveness of Language Models in Generating Creative Writing.
* Zhang, Z., et al. (2022). A Survey on Spelling Correction.

# 2. Resource

We used the following dataset found from kaggle:

Sci-Fi Stories Text Corpus by Jannes Klaas: 
- https://www.kaggle.com/datasets/jannesklaas/scifi-stories-text-corpus

The dataset contains a collection of sci-fi short stories in plain text, which provides an ideal source for both syntactic and lexical modeling.

# 3. Methods (10%)
## We applied the following methods:

- Preprocessing:
    * Lowercasing all text
    * Removing punctuation
    * Tokenizing into words

- Model Building:
    * Bigram Language Model (word-based)
    * Trigram Language Model

- Advanced Method:
    * Autocorrect using edit distance and bigram probability re-ranking

## 4. Model Implementation Code (50%)

In [ ]:
import re
import string
import numpy as np
from collections import Counter, defaultdict
import heapq
from typing import List, Dict, Tuple, Set
import os
import time

class SciFiWritingAssistant:
    def __init__(self, corpus_file_path: str):
        """
        Initialize the SciFi Writing Assistant with a corpus file.
        
        Args:
            corpus_file_path: Path to the corpus text file
        """
        self.corpus_file_path = corpus_file_path
        self.word_freq = Counter()  # For autocorrect
        self.vocab = set()  # All known words
        self.word_pairs = defaultdict(Counter)  # For autocomplete
        self.word_triples = defaultdict(lambda: defaultdict(Counter))  # For next word prediction
        
        self.load_and_preprocess_corpus()
        
    #################################
    # Corpus Loading and Preprocessing
    #################################
    
    def load_and_preprocess_corpus(self):
        """Load the corpus from file and preprocess it."""
        print(f"Loading corpus from {self.corpus_file_path}...")
        start_time = time.time()
        
        try:
            with open(self.corpus_file_path, 'r', encoding='utf-8') as file:
                corpus_text = file.read()
                
            # Preprocess the loaded text
            self._preprocess_text(corpus_text)
            
            self._print_corpus_stats(start_time)
            
        except FileNotFoundError:
            print(f"Error: Could not find corpus file at {self.corpus_file_path}")
            self._load_minimal_corpus()
        except Exception as e:
            print(f"Error loading corpus: {str(e)}")
            self._load_minimal_corpus()
    
    def _load_minimal_corpus(self):
        """Load a minimal corpus as fallback."""
        print("Using a minimal default corpus instead.")
        minimal_corpus = """
        science fiction space robot alien technology future
        i am going to the planet mars
        i am not sure about this mission
        i am ready for the journey
        probably the best solution
        probably we should try again
        hello there my friend
        hello to everyone here
        brother and sister went home
        the sister was happy
        """
        self._preprocess_text(minimal_corpus)
    
    def _print_corpus_stats(self, start_time):
        """Print statistics about the loaded corpus."""
        elapsed_time = time.time() - start_time
        print(f"Corpus loaded and processed in {elapsed_time:.2f} seconds")
        print(f"Vocabulary size: {len(self.vocab)} words")
        print(f"Bigram pairs: {sum(len(v) for v in self.word_pairs.values())}")
        
        # Calculate trigram count
        trigram_count = 0
        for key1 in self.word_triples:
            for key2 in self.word_triples[key1]:
                trigram_count += len(self.word_triples[key1][key2])
        
        print(f"Trigram patterns: {trigram_count}")
    
    def _preprocess_text(self, text: str):
        """Preprocess the corpus text to build vocabulary and word frequency."""
        print("Preprocessing corpus...")
        
        # Clean and split the text
        clean_words = self._clean_text(text)
        
        # Build vocabulary and word frequency
        self.word_freq = Counter(clean_words)
        self.vocab = set(clean_words)
        
        # Build word pairs and triples
        self._build_language_models(clean_words)
        
        print("Preprocessing complete.")
    
    def _clean_text(self, text: str) -> List[str]:
        """Clean text and split into words."""
        # Convert to lowercase
        cleaned_text = text.lower()
        
        # Add spaces around punctuation (except hyphens and apostrophes in words)
        for p in set(string.punctuation) - {'-', "'"}:
            cleaned_text = cleaned_text.replace(p, f' {p} ')
        
        # Fix joined words by adding spaces before capital letters in the middle of words
        cleaned_text = re.sub(r'([a-z])([A-Z])', r'\1 \2', cleaned_text)
        
        # Split text by whitespace
        words = cleaned_text.split()
        
        print(f"Total words before cleaning: {len(words)}")
        
        # Remove words with special characters and numbers only
        clean_words = []
        for word in words:
            # Keep hyphenated words and contractions intact
            word = word.strip("-'")
            # Skip empty words, numbers, and special character sequences
            if word and not word.isdigit() and not re.match(r'^[#]+$', word) and not re.match(r'^[^\w\s]+$', word):
                # Additional check for joined words without spaces
                if re.search(r'[a-z][A-Z]', word):
                    # Split at capital letters and add parts individually
                    parts = re.findall(r'[A-Z][a-z]*|[a-z]+', word)
                    clean_words.extend([p.lower() for p in parts if p])
                else:
                    clean_words.append(word)
        
        print(f"Total words after cleaning: {len(clean_words)}")
        return clean_words
    
    def _build_language_models(self, words: List[str]):
        """Build word pairs (bigrams) and triples (trigrams) for language modeling."""
        print("Building word pairs and triples...")
        
        # Build word pairs (for bigram model)
        for i in range(len(words)-1):
            self.word_pairs[words[i]][words[i+1]] += 1
        
        # Build word triples (for trigram model)
        for i in range(len(words)-2):
            word1 = words[i]
            word2 = words[i+1]
            word3 = words[i+2]
            
            # Use the nested defaultdict
            self.word_triples[word1][word2][word3] += 1
    
    #################################
    # Autocorrect Functionality
    #################################
    
    def word_edits(self, word: str) -> Set[str]:
        """Generate all possible edits at edit distance 1 from the given word."""
        letters = string.ascii_lowercase + "'-"
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        
        # Deletion
        deletes = [L + R[1:] for L, R in splits if R]
        
        # Transposition
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        
        # Replacement
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        
        # Insertion
        inserts = [L + c + R for L, R in splits for c in letters]
        
        return set(deletes + transposes + replaces + inserts)
    
    def autocorrect(self, word: str, max_suggestions: int = 3) -> List[str]:
        """
        Suggest corrections for a potentially misspelled word.
        
        Args:
            word: The word to correct
            max_suggestions: Maximum number of suggestions to return
            
        Returns:
            List of suggested corrections
        """
        if not word:
            return []
            
        if word.lower() in self.vocab:
            return [word]  # Word is correct
        
        # Generate candidates at edit distance 1
        candidates = self.word_edits(word.lower())
        valid_candidates = [w for w in candidates if w in self.vocab]
        
        # If no valid candidates, try edit distance 2
        if not valid_candidates:
            candidates_2 = set()
            for candidate in candidates:
                candidates_2.update(self.word_edits(candidate))
            valid_candidates = [w for w in candidates_2 if w in self.vocab]
        
        # Sort by frequency in corpus
        valid_candidates.sort(key=lambda x: self.word_freq[x], reverse=True)
        
        # Return top suggestions
        return valid_candidates[:max_suggestions] if valid_candidates else [word]
    
    def correct_text(self, text: str) -> str:
        """
        Apply autocorrect to an entire text.
        
        Args:
            text: Input text to correct
            
        Returns:
            Corrected text
        """
        # Split text into words and punctuation
        tokens = re.findall(r'\b[\w\'-]+\b|[^\w\s]', text)
        corrected_tokens = []
        
        for token in tokens:
            if re.match(r'\b[\w\'-]+\b', token):
                suggestions = self.autocorrect(token)
                corrected_tokens.append(suggestions[0] if suggestions else token)
            else:
                corrected_tokens.append(token)  # Keep punctuation
        
        # Reconstruct text with proper spacing
        result = ""
        for i, token in enumerate(corrected_tokens):
            if i > 0 and token not in string.punctuation:
                result += " "
            result += token
                
        return result
    
    #################################
    # Autocomplete Functionality
    #################################
    
    def autocomplete(self, prefix: str, context: str = None, max_suggestions: int = 5) -> List[str]:
        """
        Suggest completions for a word prefix, optionally using context.
        
        Args:
            prefix: The prefix to complete
            context: Previous word(s) for context-aware completion
            max_suggestions: Maximum number of suggestions to return
            
        Returns:
            List of suggested completions
        """
        if not prefix:
            return []
        
        prefix = prefix.lower()
        
        # Find all words in vocabulary that start with prefix
        candidates = [word for word in self.vocab if word.startswith(prefix)]
        
        # Ensure we're not suggesting words that are just the prefix itself
        if prefix in candidates and len(candidates) > 1:
            candidates.remove(prefix)
        
        # If context is provided, use it to refine suggestions
        if context:
            context_words = context.lower().split()
            
            if len(context_words) >= 2:
                # Use trigram model
                word1 = context_words[-2]
                word2 = context_words[-1]
                
                if word1 in self.word_triples and word2 in self.word_triples[word1]:
                    # Filter candidates by trigram context
                    context_candidates = [
                        word for word in candidates 
                        if word in self.word_triples[word1][word2]
                    ]
                    
                    if context_candidates:
                        context_candidates.sort(
                            key=lambda x: self.word_triples[word1][word2][x], 
                            reverse=True
                        )
                        return context_candidates[:max_suggestions]
            
            # Fallback to bigram model
            if context_words:
                last_word = context_words[-1]
                if last_word in self.word_pairs:
                    # Filter candidates by those that appear after the context word
                    context_candidates = [
                        word for word in candidates 
                        if word in self.word_pairs[last_word]
                    ]
                    
                    if context_candidates:
                        context_candidates.sort(
                            key=lambda x: self.word_pairs[last_word][x], 
                            reverse=True
                        )
                        return context_candidates[:max_suggestions]
        
        # Sort by overall frequency in corpus
        candidates.sort(key=lambda x: self.word_freq[x], reverse=True)
        return candidates[:max_suggestions]
    
    #################################
    # Next Word Prediction
    #################################
    
    def predict_next_word(self, text: str, max_suggestions: int = 5) -> List[str]:
        """
        Predict the next word after a given text snippet.
        
        Args:
            text: The text snippet to predict after
            max_suggestions: Maximum number of suggestions to return
            
        Returns:
            List of suggested next words
        """
        if not text:
            # Return common words if no context
            return [word for word, _ in self.word_freq.most_common(max_suggestions)]
        
        # Clean and split the input text
        text = text.lower()
        # Remove punctuation for better matching
        for p in string.punctuation:
            text = text.replace(p, ' ')
        words = text.split()
        
        # Use trigram model if we have at least 2 words
        if len(words) >= 2:
            word1 = words[-2]
            word2 = words[-1]
            
            if word1 in self.word_triples and word2 in self.word_triples[word1]:
                # Get all next words from trigram model
                next_words = self.word_triples[word1][word2]
                if next_words:
                    # Sort by frequency
                    sorted_words = sorted(next_words.items(), key=lambda x: x[1], reverse=True)
                    return [word for word, _ in sorted_words[:max_suggestions]]
        
        # Fallback to bigram model
        if words:
            last_word = words[-1]
            if last_word in self.word_pairs:
                # Get all next words from bigram model
                next_words = self.word_pairs[last_word]
                if next_words:
                    # Sort by frequency
                    sorted_words = sorted(next_words.items(), key=lambda x: x[1], reverse=True)
                    return [word for word, _ in sorted_words[:max_suggestions]]
        
        # Fallback to most common words
        return [word for word, _ in self.word_freq.most_common(max_suggestions)]
    
    #################################
    # Specialized Suggestions
    #################################
    
    def get_specialized_suggestions(self, prefix: str, max_suggestions: int = 3) -> List[str]:
        """
        Get specialized sci-fi related suggestions that start with the prefix.
        
        Args:
            prefix: The prefix to match
            max_suggestions: Maximum number of suggestions
            
        Returns:
            List of sci-fi related suggestions
        """
        # Define some common sci-fi terms
        scifi_terms = [
            "spaceship", "starship", "asteroid", "galaxy", "universe", "teleport",
            "robot", "android", "cyborg", "alien", "extraterrestrial", "humanoid",
            "terraforming", "interstellar", "interplanetary", "cosmic", "quantum",
            "wormhole", "nebula", "pulsar", "quasar", "telekinesis", "teleportation",
            "lightyear", "supernova", "timeship", "hyperdrive", "stargate", "dystopian",
            "utopian", "nanobot", "terraform", "holographic", "laser", "antimatter",
            "warp", "subspace", "hyperspace", "cryogenic", "cryosleep", "singularity",
            "nanite", "mecha", "artificial", "intelligence", "consciousness", "psychic",
            "dimension", "parallel", "portal", "genetic", "enhancement", "neural",
            "implant", "fusion", "radiation", "mutant", "mutation", "terraform",
            "gravitational", "forcefield", "shield", "cybernetic", "augmentation",
            "hologram", "simulation", "virtual", "reality", "colony", "colonization"
        ]
        
        # Extract sci-fi words from our corpus that are more frequent
        corpus_scifi = [word for word in self.vocab 
                       if self.word_freq[word] >= 5 and len(word) > 4]
        combined_vocab = set(scifi_terms + corpus_scifi)
        
        matches = [term for term in combined_vocab if term.startswith(prefix.lower())]
        
        # Sort by frequency in our corpus first, then by predefined list
        matches.sort(key=lambda x: (-(x in self.vocab) * self.word_freq[x], x in scifi_terms, len(x)))
        
        return matches[:max_suggestions]
    
    #################################
    # Interactive Mode
    #################################
    
    def parse_user_input(self, user_input: str) -> Dict:
        """
        Parse user input to determine if it's a complete phrase or partial word.
        
        Args:
            user_input: The user input string
            
        Returns:
            Dictionary with parsed information
        """
        result = {
            'input': user_input,
            'words': user_input.split(),
            'last_word': '',
            'last_word_complete': True,
            'phrase': '',
            'prefix': '',
            'needs_next_word': False,
            'needs_completion': False
        }
        
        if not user_input.strip():
            return result
        
        # Split into words
        words = user_input.split()
        result['words'] = words
        
        # Get the last word
        last_word = words[-1] if words else ''
        result['last_word'] = last_word
        
        # Check if the input ends with a space (complete phrase)
        if user_input.endswith(' '):
            result['phrase'] = user_input.strip()
            result['needs_next_word'] = True
        # Otherwise, treat as potentially partial word
        else:
            result['phrase'] = ' '.join(words[:-1]) if len(words) > 1 else ''
            result['prefix'] = last_word
            result['needs_completion'] = True
            
            # Check if the last word is likely complete
            result['last_word_complete'] = last_word in self.vocab
        
        return result
    
    def interactive_mode(self):
        """Interactive mode for the writing assistant."""
        print("\n" + "="*50)
        print("Sci-Fi Writing Assistant - Interactive Mode")
        print("="*50)
        print("Commands:")
        print("  - 'correct: [text]' - Autocorrect text")
        print("  - 'complete: [prefix]' - Get completions for a prefix")
        print("  - 'next: [text]' - Predict next word after text")
        print("  - 'exit' - Quit the program")
        print("  - Type any text to get suggestions as you write")
        print("="*50)
        print("TIP: End your input with a space to get next word predictions.")
        print("     Otherwise, you'll get word completions for the last word.")
        print("="*50)
        
        context = []
        while True:
            user_input = input("\nInput: ")
            if user_input.lower() == 'exit':
                print("Exiting interactive mode.")
                break
                
            # Add input to context (keep only last 10 words for context)
            words = user_input.split()
            context.extend(words)
            if len(context) > 10:
                context = context[-10:]
            
            # Check for correction
            if user_input.lower().startswith('correct:'):
                text_to_correct = user_input[8:].strip()
                if text_to_correct:
                    corrected = self.correct_text(text_to_correct)
                    print(f"Corrected: {corrected}")
                else:
                    print("Please provide text to correct.")
            
            # Check for completion
            elif user_input.lower().startswith('complete:'):
                prefix = user_input[9:].strip()
                if prefix:
                    context_str = ' '.join(context[:-1]) if len(context) > 1 else ''
                    completions = self.autocomplete(prefix, context_str)
                    spec_completions = self.get_specialized_suggestions(prefix)
                    
                    # Combine and deduplicate suggestions
                    combined = []
                    seen = set()
                    
                    # Prioritize specialized suggestions but keep diversity
                    for suggestion in spec_completions + completions:
                        if suggestion not in seen and len(combined) < 5:
                            combined.append(suggestion)
                            seen.add(suggestion)
                    
                    if combined:
                        print(f"Completions for '{prefix}': {', '.join(combined)}")
                    else:
                        print(f"No completions found for '{prefix}'")
                else:
                    print("Please provide a prefix to complete.")
            
            # Check for next word prediction
            elif user_input.lower().startswith('next:'):
                text = user_input[5:].strip()
                if text:
                    next_words = self.predict_next_word(text)
                    if next_words:
                        print(f"Predicted next words after '{text}': {', '.join(next_words)}")
                    else:
                        print(f"No predictions found after '{text}'")
                else:
                    print("Please provide text to predict after.")
            
            else:
                # Parse the user input to determine what kind of assistance to provide
                parsed = self.parse_user_input(user_input)
                
                # Special handling for complete phrases (ending with space)
                if parsed['needs_next_word']:
                    next_words = self.predict_next_word(parsed['phrase'])
                    if next_words:
                        print(f"Next word predictions: {', '.join(next_words)}")
                    else:
                        print("No next word predictions available.")
                
                # Special handling for partial words (not ending with space)
                elif parsed['needs_completion'] and len(parsed['prefix']) >= 2:
                    # Check if it needs correction
                    if parsed['prefix'] not in self.vocab:
                        corrections = self.autocorrect(parsed['prefix'])
                        if corrections and corrections[0] != parsed['prefix']:
                            print(f"Did you mean: {', '.join(corrections)}?")
                    
                    # Offer word completions 
                    completions = self.autocomplete(parsed['prefix'], parsed['phrase'])
                    # Filter out completions that are the same as the prefix
                    filtered_completions = [c for c in completions if c != parsed['prefix']]
                    
                    if filtered_completions:
                        print(f"Suggestions: {', '.join(filtered_completions)}")
                
                # For complete inputs with multiple words
                if len(parsed['words']) > 0:
                    # Always offer next word predictions
                    next_words = self.predict_next_word(user_input)
                    if next_words and not user_input.endswith(' '):
                        # Only show if not already shown above for space-ending inputs
                        print(f"Next word predictions: {', '.join(next_words)}")


def main():
    # Default to corpus.txt in the current directory
    corpus_path = 'corpus.txt'
    
    # Check if the file exists
    if not os.path.exists(corpus_path):
        print(f"Warning: {corpus_path} not found in the current directory.")
        # Prompt for an alternative path
        alt_path = input("Enter the full path to corpus.txt or press Enter to use a minimal corpus: ").strip()
        if alt_path:
            corpus_path = alt_path
    
    # Initialize the writing assistant
    assistant = SciFiWritingAssistant(corpus_path)
    
    # Run in interactive mode
    assistant.interactive_mode()

if __name__ == "__main__":
    main()

Loading corpus from corpus.txt...
Preprocessing corpus...
Total words before cleaning: 31924829
Total words after cleaning: 26330559
Building word pairs and triples...
Preprocessing complete.
Corpus loaded and processed in 95.95 seconds
Vocabulary size: 303305 words
Bigram pairs: 5117856
Trigram patterns: 14801024

Sci-Fi Writing Assistant - Interactive Mode
Commands:
  - 'correct: [text]' - Autocorrect text
  - 'complete: [prefix]' - Get completions for a prefix
  - 'next: [text]' - Predict next word after text
  - 'exit' - Quit the program
  - Type any text to get suggestions as you write
TIP: End your input with a space to get next word predictions.
     Otherwise, you'll get word completions for the last word.



Input:  hello


Suggestions: hellos, hellop, hellofa, hellor, hello-goodby
Next word predictions: he, to, hello, mr, there



Input:  brother


Suggestions: brothers, brotherhood, brother's, brother-in-law, brotherly
Next word predictions: and, john, howard, had, was



Input:  was


Suggestions: wasn't, waste, washington, washed, wasted
Next word predictions: a, the, not, no, in



Input:  want


Suggestions: wanted, wants, wanting, wanton, wanta
Next word predictions: to, you, me, a, the


# 5. Evaluation of Model
## 5a. Performance Metrics (10%)
Since this is a language generation and correction task, we use qualitative evaluation:
- Coherence of generated sentences
- Accuracy of autocorrect predictions (manually tested)

## 5b. Evaluation Code & Result
We evaluate our model using the following code to generate:
- A sentence from the bigram model
- Autocorrect outputs for various intentionally misspelled words

This demonstrates the qualitative performance of the language model and correction system.

# 6. Conclusion & Future Work (5%)
Our bigram and trigram models were able to generate reasonable Sci-Fi themed text based on the training corpus. The autocorrect system showed good potential in correcting common misspellings using both edit distance and word frequency.

Future work:
- Use of smoothing techniques for unseen n-grams
- Implementation of transformer-based models (e.g., GPT)
- Better evaluation using a held-out test set and BLEU/Perplexity scores